In [1]:
import subprocess
import fenics as fe
from dolfin import Function
from fenics import near, between, Function, TestFunction, TrialFunction, grad, div, det, tr, dot

from gmsh_interface import GmshInterface

In [2]:
def convert_msh_to_xml(fn_msh, fn_xml):
    subprocess.call("dolfin-convert ../geo/%s ../geo/%s" % (fn_msh, fn_xml))


class Hyperplane(fe.SubDomain):
    def __init__(self,z_value):
        self.z_value = z_value

    def inside(self, x, on_boundary):
        return on_boundary and near(x[2], self.z_value)


class Stripe(fe.SubDomain):
    def __init__(self,z_interval):
        self.z_interval = z_interval

    def inside(self, x, on_boundary):
        return on_boundary and between(x[2], self.z_interval[0], self.z_interval[1])


class MuscleState:

    def __init__(self):
        self.mesh = None

        # self.domains = None
        # self.boundaries = None

        self.dx = None
        self.ds = None

        # current deformation
        self.u = None

    def load_mesh(self, fn: str = None):
        if fn is not None:
            self.mesh = fe.Mesh(fn)
        else:
            self.mesh = fe.UnitCubeMesh(10,12,14)



    def compute_static_deformation(self):

        assert self.mesh is not None

        # now we define subdomains on the mesh

        bottom = fe.CompiledSubDomain('near(x[2], 0) && on_boundary')
        top = fe.CompiledSubDomain('near(x[2], 1) && on_boundary')
        # middle = fe.CompiledSubDomain('x[2] > 0.3 && x[2] < 0.7')

        # Initialize mesh function for interior domains
        self.domains = fe.MeshFunction('size_t', self.mesh, 3)
        self.domains.set_all(0)
        # middle.mark(self.domains, 1)

        # Initialize mesh function for boundary domains
        self.boundaries = fe.MeshFunction('size_t', self.mesh, 2)
        self.boundaries.set_all(0)
        bottom.mark(self.boundaries, 1)
        top.mark(self.boundaries, 2)

        # Define new measures associated with the interior domains and
        # exterior boundaries

        self.dx = fe.Measure('dx', domain=self.mesh, subdomain_data=self.domains)
        self.ds = fe.Measure('ds', domain=self.mesh, subdomain_data=self.boundaries)

        # define function spaces
        V = fe.VectorFunctionSpace(self.mesh, "Lagrange", 1)
        # now we define subdomains on the mesh

        bottom = fe.CompiledSubDomain('near(x[2], 0) && on_boundary')
        top = fe.CompiledSubDomain('near(x[2], 1) && on_boundary')
        # middle = fe.CompiledSubDomain('x[2] > 0.3 && x[2] < 0.7')

        d = self.mesh.geometry().dim()

        # Initialize mesh function for interior domains
        self.domains = fe.MeshFunction('size_t', self.mesh, d)
        self.domains.set_all(0)
        # middle.mark(self.domains, 1)

        # Initialize mesh function for boundary domains
        self.boundaries = fe.MeshFunction('size_t', self.mesh, d - 1)
        self.boundaries.set_all(0)
        bottom.mark(self.boundaries, 1)
        top.mark(self.boundaries, 2)

        # Define new measures associated with the interior domains and
        # exterior boundaries

        self.dx = fe.Measure('dx', domain=self.mesh, subdomain_data=self.domains)
        self.ds = fe.Measure('ds', domain=self.mesh, subdomain_data=self.boundaries)

        c_zero = fe.Constant((0,0,0))

        # define boundary conditions
        bc_bottom = fe.DirichletBC(V, c_zero, bottom)
        bc_top    = fe.DirichletBC(V, c_zero, top)

        bcs = [bc_bottom]  # , bc_top]

        # define functions
        du = TrialFunction(V)
        v = TestFunction(V)
        u = Function(V)
        B = fe.Constant((0., 2.0, 0.))
        T = fe.Constant((0.0, 0.0, 0.0))

        d = u.geometric_dimension()
        I = fe.Identity(d)
        F = I + grad(u)
        C = F.T*F

        I_1 = tr(C)
        J = det(F)

        E, mu = 10., 0.3
        mu, lmbda = fe.Constant(E/(2*(1+mu))), fe.Constant(E*mu/((1+mu)*(1-2*mu)))

        # stored energy (comp. neo-hookean model)
        psi = (mu/2.)*(I_1 - 3) - mu*fe.ln(J) + (lmbda/2.)*(fe.ln(J))**2

        dx = self.dx
        ds = self.ds

        Pi = psi*fe.dx - dot(B, u)*fe.dx - dot(T, u)*fe.ds

        F = fe.derivative(Pi, u, v)
        J = fe.derivative(F, u, du)

        fe.solve(F == 0, u, bcs, J=J)

        # save results
        self.u = u
        
        # write to disk
        output = fe.File("/tmp/static.pvd")
        output << u


g = GmshInterface("../geo/simple_muscle_3d.geo")
g.set_parameter("lc",0.08)
#for i in [1,2,3]:
#    g.set_parameter("T_"+str(i),0.4)
#    g.set_parameter("W_"+str(i),0.4)
g.generate_xml("../geo/test.xml")

import matplotlib.pyplot as plt

FileNotFoundError: [Errno 2] No such file or directory: '../geo/simple_muscle_3d.geo'

In [3]:
m = MuscleState()
m.load_mesh("../geo/test.xml")
fe.plot(m.mesh)
plt.show()
m.compute_static_deformation()

RuntimeError: boost::filesystem::create_directory: Permission denied: "../geo"

In [4]:
from dolfin import *

In [7]:
# Create empty Mesh
mesh = Mesh()

In [8]:
# Create list of polygonal domain vertices
domain_vertices = [Point(0.0, 0.0),
                   Point(10.0, 0.0),
                   Point(10.0, 2.0),
                   Point(8.0, 2.0),
                   Point(7.5, 1.0),
                   Point(2.5, 1.0),
                   Point(2.0, 4.0),
                   Point(0.0, 4.0),
                   Point(0.0, 0.0)]

In [9]:
# Generate mesh and plot
PolygonalMeshGenerator.generate(mesh, domain_vertices, 0.25);
plot(mesh, interactive=True)

NameError: name 'PolygonalMeshGenerator' is not defined

In [10]:
# Generate 3D mesh from OFF file input (tetrahedron)
PolyhedralMeshGenerator.generate(mesh, "../tetrahedron.off", 0.05)
plot(mesh, interactive=True)

# Generate 3D mesh from OFF file input (cube)
PolyhedralMeshGenerator.generate(mesh, "../cube.off", 0.05)
plot(mesh, interactive=True)

NameError: name 'PolyhedralMeshGenerator' is not defined

In [11]:
from fenics import *
import numpy as np

T = 2.0            # final time
num_steps = 10     # number of time steps
dt = T / num_steps # time step size
alpha = 3          # parameter alpha
beta = 1.2         # parameter beta

# Create mesh and define function space
nx = ny = 8
mesh = UnitSquareMesh(nx, ny)
V = FunctionSpace(mesh, 'P', 1)

# Define boundary condition
u_D = Expression('1 + x[0]*x[0] + alpha*x[1]*x[1] + beta*t',
                 degree=2, alpha=alpha, beta=beta, t=0)

def boundary(x, on_boundary):
    return on_boundary

bc = DirichletBC(V, u_D, boundary)

# Define initial value
u_n = interpolate(u_D, V)
#u_n = project(u_D, V)

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = Constant(beta - 2 - 2*alpha)

F = u*v*dx + dt*dot(grad(u), grad(v))*dx - (u_n + dt*f)*v*dx
a, L = lhs(F), rhs(F)

# Time-stepping
u = Function(V)
t = 0
for n in range(num_steps):

    # Update current time
    t += dt
    u_D.t = t

    # Compute solution
    solve(a == L, u, bc)

    # Plot solution
    plot(u)

    # Compute error at vertices
    u_e = interpolate(u_D, V)
    error = np.abs(u_e.vector().array() - u.vector().array()).max()
    print('t = %.2f: error = %.3g' % (t, error))

    # Update previous solution
    u_n.assign(u)

# Hold plot
interactive()

Unknown ufl object type FiniteElement


Exception: Unknown ufl object type FiniteElement